## Sales Prediction


The implementation of our model will have 3 steps:

1) Data Wrangling

2) Data Transformation to make it stationary and supervised

3) Building the LSTM model & evaluation

### 1) Data Wrangling

In [1]:
from __future__ import division
from datetime import datetime, timedelta,date

import sys
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Installing packages
#!pip install plotly --user 
#!pip install cufflinks==0.8.2 --user

In [3]:
#importing Libraries
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as pyoff
import cufflinks as cf
import plotly.graph_objs as go



/opt/conda/envs/Python36/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:558: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


/opt/conda/envs/Python36/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:531: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis




In [4]:
#importing Keras
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

Using TensorFlow backend.


In [5]:
#initiate plotly
pyoff.init_notebook_mode(connected=True)

In [6]:
import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

In [7]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0




,order_id,client_id,service_id,gift_id,gift_cost,total_gifts,total_amount,fee,variable_cost,total_revenue,month,year,date
0,11110,419,1,16,1289.54,13,16764,3352.8,4326,20116.8,3,2018,3-2018
1,11111,503,3,21,235.66,50,11783,2356.6,5976,14139.6,10,2019,10-2019
2,11112,395,2,23,211.97,71,15050,3010.0,4836,18060.0,3,2019,3-2019
3,11113,292,1,22,445.24,38,16919,3383.8,6144,20302.8,6,2019,6-2019
4,11114,408,1,14,1247.91,11,13727,2745.4,5632,16472.4,4,2018,4-2018
5,11115,204,2,17,720.29,24,17287,3457.4,6002,20744.4,10,2018,10-2018
6,11116,401,1,24,1116.27,15,16744,3348.8,4934,20092.8,7,2018,7-2018
7,11117,207,3,17,260.98,40,10439,2087.8,5544,12526.8,8,2019,8-2019
8,11118,481,3,16,162.03,63,10208,2041.6,5148,12249.6,9,2019,9-2019
9,11119,274,2,18,149.58,76,11368,2273.6,5563,13641.6,8,2018,8-2018


In [8]:
#convert date field from string to datetime
df_sales['date'] = pd.to_datetime(df_sales['date'])

#### Forecasted monthly total sales. We aggregated the data at the monthly level and summed up the total_revenue column.

In [9]:
#represent month in date field as its first day
df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
df_sales['date'] = pd.to_datetime(df_sales['date'])
#groupby date and sum the sales
df_sales = df_sales.groupby('date').total_revenue.sum().reset_index()

In [10]:
df_sales.head(20)

,date,total_revenue
0,2018-01-01,2724585.6
1,2018-02-01,3261540.0
2,2018-03-01,2878356.0
3,2018-04-01,3180222.0
4,2018-05-01,3122652.0
5,2018-06-01,2836056.0
6,2018-07-01,2806626.0
7,2018-08-01,2873586.0
8,2018-09-01,3054661.2
9,2018-10-01,2999848.8


### 2) Data Transformation

To model the forecast easier and accurate, we will do the below transformations:
- Convert the data to stationary if it is not
- Converting from time series to supervised for having the feature set of the LSTM model
- Scale the data

In [11]:
#plot monthly sales
plot_data = [
    go.Scatter(
        x=df_sales['date'],
        y=df_sales['total_revenue'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)



- Obviously, it is not stationary and has an increasing trend over the months.
- One method is to get the difference in sales compared to the previous month and build the model on it

In [12]:
#create a new dataframe to model the difference
df_diff = df_sales.copy()
#add previous sales to the next row
df_diff['prev_total_revenue'] = df_diff['total_revenue'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['total_revenue'] - df_diff['prev_total_revenue'])
df_diff.head(20)

,date,total_revenue,prev_total_revenue,diff
1,2018-02-01,3261540.0,2724585.6,536954.4
2,2018-03-01,2878356.0,3261540.0,-383184.0
3,2018-04-01,3180222.0,2878356.0,301866.0
4,2018-05-01,3122652.0,3180222.0,-57570.0
5,2018-06-01,2836056.0,3122652.0,-286596.0
6,2018-07-01,2806626.0,2836056.0,-29430.0
7,2018-08-01,2873586.0,2806626.0,66960.0
8,2018-09-01,3054661.2,2873586.0,181075.2
9,2018-10-01,2999848.8,3054661.2,-54812.4
10,2018-11-01,3000444.0,2999848.8,595.2


In [13]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['date'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

- From the above graph, it has trends over the months and not stationary


We built the feature set and used previous monthly sales data to forecast the next ones. The look-back period may vary for every model. We used 12 for this example.
So we created columns from lag_1 to lag_12 and assign values by using shift() method:

In [14]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_total_revenue'],axis=1)
#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [15]:
df_supervised.head(20)

,date,total_revenue,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2019-02-01,2679206.4,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0,-29430.0,-286596.0,-57570.0,301866.0,-383184.0,536954.4
1,2019-03-01,3300183.6,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0,-29430.0,-286596.0,-57570.0,301866.0,-383184.0
2,2019-04-01,2819240.4,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0,-29430.0,-286596.0,-57570.0,301866.0
3,2019-05-01,3262809.6,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0,-29430.0,-286596.0,-57570.0
4,2019-06-01,2958664.8,-304144.8,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0,-29430.0,-286596.0
5,2019-07-01,3089629.2,130964.4,-304144.8,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0,-29430.0
6,2019-08-01,3316816.8,227187.6,130964.4,-304144.8,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2,66960.0
7,2019-09-01,2804488.8,-512328.0,227187.6,130964.4,-304144.8,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4,181075.2
8,2019-10-01,3064311.6,259822.8,-512328.0,227187.6,130964.4,-304144.8,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2,-54812.4
9,2019-11-01,3053448.0,-10863.6,259822.8,-512328.0,227187.6,130964.4,-304144.8,443569.2,-480943.2,620977.2,-385072.8,40224.0,23611.2,595.2


### How useful are our features for prediction?


Adjusted R-squared is the answer. It tells us how good our features explain the variation in our label (lag_1 to lag_12 for diff, in our example).


So what happened above?

Basically, we fit a linear regression model (OLS — Ordinary Least Squares) and calculate the Adjusted R-squared. For the example above, we just used lag_1 to see how much it explains the variation in column diff. The output of this code block is:

In [16]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.5924748744718605


lag_1 explains 60% of the variation

In [17]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.6308609866939294


In [18]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.7409265525598578


Adding one more feature increased the score from 60% to 74%.




The result is not so impressive as the score is 74%. Now we can build the model after scaling our data. But there is one more step before scaling. So We split our data into train and test sets. As the test set, we have selected the last 6 months’ sales

In [19]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['total_revenue','date'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

As the scaler, we are going to use MinMaxScaler, which will scale each future between -1 and 1:

In [20]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

### 3) Building the LSTM model

In [21]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [22]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=300, batch_size=1, verbose=1, shuffle=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



Epoch 1/300
5/5 [==============================] - 2s 323ms/step - loss: 0.4926
Epoch 2/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4968
Epoch 3/300
5/5 [==============================] - 0s 11ms/step - loss: 0.4944
Epoch 4/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4883
Epoch 5/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4816
Epoch 6/300
5/5 [==============================] - 0s 11ms/step - loss: 0.4747
Epoch 7/300
5/5 [==============================] - 0s 12ms/step - loss: 0.4678
Epoch 8/300
5/5 [==============================] - 0s 11ms/step - loss: 0.4609
Epoch 9/300
5/5 [==============================] - 0s 8ms/step - loss: 0.4540
Epoch 10/300
5/5 [==============================] - 0s 12ms/step - loss: 0.4471
Epoch 11/300
5/5 [==============================] - 0s 9ms/step - loss: 0.4402
Epoch 12/300
5/5 [==============================] - 0s 11ms/step - loss: 0.4333
Epoch 13/300
5/5 [==============================] - 0

5/5 [==============================] - 0s 8ms/step - loss: 0.0299
Epoch 105/300
5/5 [==============================] - 0s 11ms/step - loss: 0.0288
Epoch 106/300
5/5 [==============================] - 0s 12ms/step - loss: 0.0276
Epoch 107/300
5/5 [==============================] - 0s 8ms/step - loss: 0.0265
Epoch 108/300
5/5 [==============================] - 0s 12ms/step - loss: 0.0254
Epoch 109/300
5/5 [==============================] - 0s 12ms/step - loss: 0.0244
Epoch 110/300
5/5 [==============================] - 0s 8ms/step - loss: 0.0234
Epoch 111/300
5/5 [==============================] - 0s 12ms/step - loss: 0.0224
Epoch 112/300
5/5 [==============================] - 0s 11ms/step - loss: 0.0214
Epoch 113/300
5/5 [==============================] - 0s 62ms/step - loss: 0.0205
Epoch 114/300
5/5 [==============================] - 0s 10ms/step - loss: 0.0197
Epoch 115/300
5/5 [==============================] - 0s 13ms/step - loss: 0.0188
Epoch 116/300
5/5 [==========================

The code block above prints how the model improves itself and reduce the error in each epoch:

In [23]:
y_pred = model.predict(X_test,batch_size=1)
y_pred

array([[ 1.1960807 ],
       [-0.09633537],
       [ 1.5373584 ],
       [ 0.33192813],
       [ 1.6807859 ],
       [ 0.77016306]], dtype=float32)

In [24]:
y_test

array([[ 0.11062033],
       [ 0.2852667 ],
       [-1.05696382],
       [ 0.34450002],
       [-0.14679935],
       [-0.34586346]])

#### How we can see the actual sales prediction?


First, we need to do the inverse transformation for scaling:

In [25]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    p=np.concatenate([y_pred[index],X_test[index]],axis=1)
    pred_test_set.append(p)
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [26]:
p

array([[ 0.77016306, -0.14679935,  0.34450002, -1.25297987,  1.8792147 ,
         0.57512985, -3.11399327,  3.22558541, -1.83112751,  2.88124477,
        -1.33469213,  0.23613751, -0.11579562]])

Second, we need to build the dataframe has the dates and the predictions. Transformed predictions are showing the difference. We should calculate the predicted sales numbers:

In [27]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-7:].date)
act_sales = list(df_sales[-7:].total_revenue)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

df_result

,date,pred_value
0,2019-07-01,3687674
1,2019-08-01,3106569
2,2019-09-01,4233857
3,2019-10-01,3057384
4,2019-11-01,4060374
5,2019-12-01,3547794


#### Actual vs predicted:

In [28]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['total_revenue'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)